In [28]:
import numpy as np
import plotly as plt

In [29]:
import plotly.express as px

In [30]:
# Parameters

# General
t = 0
T = 300
dt = T / 100
np.random.seed(100)

# Price
mu = 0
sigma = 0.1
P_0 = 45

# Spread
intensity = 1

# Cash and inventory
Y_0 = 0
X_0 = 0

# Price process

In [31]:
def price_sim(mu, P_0, t, sigma, W):
    return P_0 + mu * t + sigma * W

# Spread

In [32]:
def poisson_point_process_sim(intensity, T):

    N = np.random.poisson(intensity * T)
    T_i = np.cumsum(np.random.exponential(1 / intensity, N))    

    return (N, T_i)

In [33]:
def spread_sim(initial_state, spread_values, T, matrix, N): 
    
    U = np.random.uniform(0, 1, 1000)
    matrix_sum = np.cumsum(matrix, axis = 1)
    s = [initial_state]
    for i in range(1, N):
        potential_position = np.sum(U[i] > matrix_sum[s[i - 1], ])
        if potential_position == s[i - 1] or potential_position == 6:
            s.append(potential_position - 1)
        else:
            s.append(potential_position)
            
    return s

In [34]:
# Spread specific objects

prob_matrix = np.array([[0, 0.41, 0.22, 0.16, 0.142, 0.065], 
                        [0.201, 0, 0.435, 0.192, 0.103, 0.067],
                        [0.113, 0.221, 0, 0.4582, 0.147, 0.059],
                        [0.07, 0.085, 0.275, 0, 0.465, 0.102],
                        [0.068, 0.049, 0.073, 0.363, 0, 0.446],
                        [0.077, 0.057, 0.059, 0.112, 0.692, 0]])
                        
spread_values = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]


In [35]:
PPP_spread = poisson_point_process_sim(intensity, T)
spread_simulated = spread_sim(0, spread_values, T, prob_matrix, PPP_spread[0])
s_t = [spread_values[i] for i in spread_simulated]

W_t = np.cumsum(np.concatenate(([0], np.random.normal(0, 1, len(spread_simulated) - 1))))

In [36]:
len(s_t)

302

In [37]:
len(spread_simulated)

302

In [38]:
fig = px.line(s_t)
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Spread evolution", title_x = 0.5)
fig.show()

In [39]:
fig = px.line(spread_simulation[1])
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Spread evolution", title_x = 0.5)
fig.show()

NameError: name 'spread_simulation' is not defined

In [ ]:
fig = px.line(W_t)
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Evolution of price over time", title_x = 0.5)
fig.show()

In [ ]:
fig = px.line(W_t)
fig.update_layout(title_text = "W_t", title_x = 0.5)
fig.update_layout(
    {
        "paper_bgcolor": "rgba(1, 20, 0, 1)",
        "plot_bgcolor": "rgba(1, 0, 0, 1)",
    }
)

fig.show()

# Execution of orders

In [ ]:
def orders_simulaton(intensity_matrix, spread, lambda_max, initial_intensities, q):

    lambda_process = []

    for t, i in zip(initial_intensities, range(len(initial_intensities) + 1)):
        spread_position = np.sum(t >= spread)  # checking the spread at the arrival time t
        lambda_process.append(intensity_matrix[spread[spread_position - 1], q[i]])   # choosing the realisation of intensity process based on the estimated intensity function and simulated spread


    acceptance = np.where(lambda_process / lambda_max < np.random.uniform(0, 1, len(initial_intensities)), False, True)  # accepted arrival times

    arrival_times = initial_intensities[acceptance]  # final arrival times

    return arrival_times

In [ ]:
intensity_matrix_a = np.array([[0.0539, 0.1485], 
                              [0.0465, 0.0979],
                              [0.0401, 0.0846],
                              [0.0360, 0.0856],
                              [0.0435, 0.1009],
                              [0.0554, 0.1202]])

intensity_matrix_b = np.array([[0.0718, 0.1763], 
                              [0.0520, 0.1144],
                              [0.0419, 0.0915],
                              [0.0409, 0.0896],
                              [0.0452, 0.0930],
                              [0.0614, 0.1255]])

intensity_matrix_sym = (intensity_matrix_a + intensity_matrix_b) / 2    # Making bid and ask sides symmetric

In [ ]:
lambda_max = np.max(intensity_matrix_sym)  # maximum lambda serving as a reference for acceptance probability
intensity_orders_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity

q_a = np.random.binomial(1, 0.5, len(intensity_orders_simulation[1]))  # ad-hoc choice of quotes for ask
q_b = np.random.binomial(1, 0.5, len(intensity_orders_simulation[1]))  # ad-hoc choice of quotes for bid

In [ ]:
orders_a = orders_simulaton(intensity_matrix_sym, spread_simulated, lambda_max, intensity_orders_simulation[1], q_a)
orders_b = orders_simulaton(intensity_matrix_sym, spread_simulated, lambda_max, intensity_orders_simulation[1], q_b)

# Cash and inventory

In [ ]:
def inventory_sim(N_a, N_b, L_a, L_b, Y_0):

    Y = [Y_0]
    index_a = 0   # counter for sizes of ask orders 
    index_b = 0   # counter for sizes of bid orders 

    for t, index_y in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):      # merged times of orders
        
        # check whether at a given time it is a bid or an ask order 
        if t in N_b:        
            Y.append(Y[index_y - 1] + l_b[index_b])   # if bid - add inventory
            index_b =+ 1
        else:
            Y.append(Y[index_y - 1] - l_a[index_a])   # if ask - subtract inventory
            index_a =+ 1

    return Y

In [ ]:
def cash_sim(N_a, N_b, L_a, L_b, X_0, q_a, q_b, s, p):

    l_a = np.random.uniform(0, 100, len(N_a)) # ad-hoc choice of size of order for ask side
    l_b = np.random.uniform(0, 100, len(N_b)) # ad-hoc choice of size of order for bid side

    X = [X_0]
    index_a = 0
    index_b = 0

    for t, index_x in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):

        # check whether at a given time it is a bid or an ask order 
        if t in N_b:
            X.append(Y[index_y - 1] + l_b[index_b])
            index_b =+ 1
        else:
            X.append(Y[index_y - 1] - l_a[index_a])
            index_a =+ 1
    return X

In [ ]:
l_a = np.random.uniform(0, 100, len(N_a)) # ad-hoc choice of sizes of orders for ask side
l_b = np.random.uniform(0, 100, len(N_b)) # ad-hoc choice of sizes of orders for bid side

In [ ]:
Y = inventory_sim(orders_a, orders_b, l_a, l_b, Y_0)
X = cash_sim(orders_a, orders_b, l_a, l_b, X_0, q_a, q_b, s_t, p_t)

In [ ]:
W_t = np.cumsum(np.concatenate(([0], np.random.normal(0, 1, len(PPP_spread[1]) - 1))))